In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from collections import Counter
import joblib
import random

In [ ]:
DATASET_PATH = "./dataset_padang_food"
IMG_SIZE = (128, 128)
RANDOM_STATE = 42

TARGET_CLASSES = [
    "ayam_pop",
    "ayam_goreng",
    "daging_rendang",
    "gulai_tunjang",
    "telur_dadar"
]

AUGMENT = True
AUG_PER_IMAGE = 2   

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [ ]:
def augment_image(img):
    aug_images = []

    h, w = img.shape[:2]

    aug_images.append(cv2.flip(img, 1))

    angle = random.uniform(-10, 10)
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    aug_images.append(cv2.warpAffine(img, M, (w, h)))

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = hsv.astype(np.float32)
    hsv[..., 2] *= random.uniform(0.8, 1.2)
    hsv[..., 2] = np.clip(hsv[..., 2], 0, 255)
    aug_images.append(cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR))

    return aug_images

In [ ]:
def extract_feature(img):
    img = cv2.resize(img, IMG_SIZE)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    hog_feat = hog(
        gray,
        orientations=9,
        pixels_per_cell=(32, 32),
        cells_per_block=(2, 2),
        block_norm="L2-Hys"
    )

    color_hist = cv2.calcHist(
        [img],
        [0, 1, 2],
        None,
        [8, 8, 8],
        [0, 256, 0, 256, 0, 256]
    )
    cv2.normalize(color_hist, color_hist)
    color_hist = color_hist.flatten()

    radius = 2
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius, method="uniform")
    lbp_hist, _ = np.histogram(
        lbp.ravel(),
        bins=np.arange(0, n_points + 3),
        range=(0, n_points + 2)
    )
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)

    feat = np.hstack([
        hog_feat * 0.45,
        color_hist * 0.30,
        lbp_hist * 0.25
    ])

    return feat

In [ ]:
X, y = [], []

print("Target classes:", TARGET_CLASSES)

for label in TARGET_CLASSES:
    class_dir = os.path.join(DATASET_PATH, label)

    if not os.path.isdir(class_dir):
        print(f"[WARNING] Folder tidak ditemukan: {label}")
        continue

    print(f"Processing class: {label}")
    for file in os.listdir(class_dir):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(class_dir, file)
            img = cv2.imread(img_path)
            if img is None:
                continue

            X.append(extract_feature(img))
            y.append(label)

            if AUGMENT:
                aug_imgs = augment_image(img)
                random.shuffle(aug_imgs)
                for aug_img in aug_imgs[:AUG_PER_IMAGE]:
                    X.append(extract_feature(aug_img))
                    y.append(label)

X = np.array(X)
y = np.array(y)

print("\nTotal samples after augmentation:", len(X))
print("Class distribution:", Counter(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

In [ ]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95, random_state=RANDOM_STATE)),
    ("svm", SVC(
        C=10,
        kernel="rbf",
        gamma="scale",
        class_weight="balanced",
        probability=True
    ))
])

In [ ]:
print("\nTraining SVM with augmentation...")
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("\nAccuracy:", acc)
print(classification_report(y_test, y_pred))

In [ ]:
joblib.dump(model, "padangfood_svm_5class_augmented.pkl")
print("\nModel saved as padangfood_svm_5class_augmented.pkl")